In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import os
import sys
import dateutil.parser

from __future__ import print_function

In [2]:
sys.path.insert(0, os.path.abspath('../..'))

from UKMovementSensing import prepacc, tud

In [3]:
execfile('config.py')

## Read in wearcodes

This file is our central administration file: it contains the filenames for all collected data.

In [4]:
wearcodes = prepacc.load_wearcodes(wearcodes_path)

## Process the accelerometer data

We will split the accelerometer data in completely valid subsets, and switch the position variables if necesssary.

The input data should contain at least folowing columns:
* timestamp
* invalid
* heuristic
* acceleration
* anglex
* angley
* angley

Other variables are also allowed. This should be the output of a GGIR process.

In [118]:
prepacc.process_data_onebyone(wearcodes, accelerometer_5sec_path, subset_path)

switched dataset with median 29.307547
switched dataset with median 30.527767
switched dataset with median 28.158655
switched dataset with median 12.313824
switched dataset with median 18.418094
switched dataset with median 34.877218
switched dataset with median 53.758234
switched dataset with median 38.752736
switched dataset with median 19.638931
switched dataset with median 20.317434
switched dataset with median 34.185083
switched dataset with median 31.965390
switched dataset with median 36.133421
File not found:  /media/sf_VBox_Shared/London/UCL_data_March/run_2017march21/accelerometer_5second/__020791_2015-04-07 11-00-18.bin_day1.csv
File not found:  /media/sf_VBox_Shared/London/UCL_data_March/run_2017march21/accelerometer_5second/__020791_2015-04-07 11-00-18.bin_day2.csv
switched dataset with median 34.811131
switched dataset with median 41.665654
File not found:  /media/sf_VBox_Shared/London/UCL_data_March/run_2017march21/accelerometer_5second/__020888_2015-03-06 09-26-54.bin_d

## Read the tud file

In [5]:
print('Process annotations...')
annotations = tud.process_annotations(annotations_path)

Process annotations...
Multiple end times per slot. first 10:
slot
126    [01:00:00, 02:00:00]
127    [01:10:00, 02:10:00]
128    [01:20:00, 02:20:00]
129    [01:30:00, 02:30:00]
130    [01:40:00, 02:40:00]
131    [01:50:00, 02:50:00]
132    [02:00:00, 03:00:00]
133    [02:10:00, 03:10:00]
134    [02:20:00, 03:20:00]
135    [02:30:00, 03:30:00]
136    [02:40:00, 03:40:00]
137    [02:50:00, 03:50:00]
138    [03:00:00, 04:00:00]
139    [03:10:00, 04:10:00]
140    [03:20:00, 04:20:00]
141    [03:30:00, 04:30:00]
142    [03:40:00, 04:40:00]
143    [03:50:00, 04:50:00]
144    [04:00:00, 05:00:00]
Name: end_time_time, dtype: object
Not all days have 144 slots! first 5:
accSmallID  day
7a1fd48f    1      137
7a2bbb74    1      118
            2       90
7a2e1cd4    1      138
7a44256e    2       72
Name: slot, dtype: int64


In [6]:
print('Join wearcodes...')
annotations_codes = tud.join_wearcodes(wearcodes, annotations)

Join wearcodes...


## Merge accelerometer data with diary

In [15]:
byName = annotations_codes.groupby(['binFile', 'day'])

In [16]:
#TODO: Slot is not correct here.

if not os.path.exists(merged_path):
    os.makedirs(merged_path)
for fn in os.listdir(subset_path):
    day = int(fn[-5])
    binFile = fn[2:-9]
    df = prepacc.load_acceleromater_data(os.path.join(subset_path, fn))
    if (binFile, day) in byName.groups:
        annotations_group = byName.get_group((binFile, day))
    else:
        # Make empty annotation file
        print('Warning: no annotations for {} day {}'.format(binFile, day))
        annotations_group = annotations_codes[0:0]
    df_annotated = tud.add_annotations(df, annotations_group)
    nr_none = len(df_annotated[df_annotated['activity'] == None])
    if nr_none > 0:
        print('Warning: {} levels have no annotation'.format(nr_none))
    df_annotated.to_csv(os.path.join(merged_path, fn), date_format='%Y-%m-%dT%H:%M:%S%z')